# RamDAQ QC report (SE)

## 1. Preparation (Just ignore this section!)
In this section, we prepare for visualization of QC results.

### Directory setting

In [ ]:
### working Dir (QCdata parent dir : "output_" + Project_id)
workdir = "/data"

### RamDA-QC-pipeline Dir
scriptdir = "/accessory/"

In [ ]:
setwd(workdir)
getwd()

source(paste0(scriptdir, "/00_sampleQC_function_nbconvert.R"))
outdir = ""

### Loading 'sample blacklist' (list of samples (cells) to be excluded from QC)

If the 'sample blacklist' is provided, samples (cells) in the list are excluded from QC results.

In [ ]:
exclude_samplelist_path = paste0(workdir, "/exclude_samplelist.txt")

if (file.exists(exclude_samplelist_path)){
  exclude_samplename = scan(exclude_samplelist_path, what=character(), sep="\n", blank.lines.skip = F)
  exclude_samplename = exclude_samplename[exclude_samplename != ""]
  exclude_samplename
} else {
  cat("exclude samplelist not found.")
  exclude_samplename = ""
}

### Loading QC result files

#### FastQC results

In [ ]:
fastqc_SE_001 = read.table(paste0(workdir, "/summary_fastQC_result.txt"), sep="\t", comment.char = "", header=T, stringsAsFactors=F)

### edit data table
fastqc_SE_001 = trim_dataset(fastqc_SE_001)
head(fastqc_SE_001)

#### RSeQC results
##### >> readDistribution

In [ ]:
### unstranded
readDist_SE_001 = read.table(paste0(workdir, "/summary_RSeQC_ReadDist_results_SE_sort.txt"), sep="\t", comment.char = "", header=T, stringsAsFactors=F)

### edit data table
readDist_SE_001 = trim_dataset(readDist_SE_001)
head(readDist_SE_001)

##### >> geneBC

In [ ]:
### unstranded
geneBC_SE_001 = read.table(paste0(workdir, "/summary_RSeQC_geneBC_results_SE_sort.txt"), sep="\t", comment.char = "", header=T, stringsAsFactors=F)

### edit data table
geneBC_SE_001 = trim_dataset(geneBC_SE_001)
head(geneBC_SE_001)

##### >> infer_experiment

In [ ]:
infer_SE_001 = read.table(paste0(workdir, "/summary_RSeQC_inferexperiment_results_SE_sort.txt"), sep="\t", comment.char = "", header=T, stringsAsFactors=F)

### edit data table
infer_SE_001 = trim_dataset(infer_SE_001)
head(infer_SE_001)

#### High-sensitivity rRNA QC results

In [ ]:
highsensitivity_rrna_result_path = paste0(workdir, "/highsensitivity_rrnaQC_summary/featureCounts.hisat2_G.T.R_loose_mapped.summary_simple.txt")

if (file.exists(highsensitivity_rrna_result_path)){
  highsenst_rrna_SE_001 = read.table(highsensitivity_rrna_result_path, sep="\t", comment.char = "", header=T, stringsAsFactors=F)

  ### edit data table
  highsenst_rrna_SE_001 = trim_dataset(highsenst_rrna_SE_001)
  head(highsenst_rrna_SE_001)
} else {
  cat("highsensitivity rrnaQC results not found.")
}

#### featureCounts results

##### >> featureCounts mRNA (transcript-level) results

In [ ]:
fcount_SE_mrna_Ts_001 = read.table(paste0(workdir, "/summary_featurecounts_results_gencode_mrna_transcript.txt"), sep="\t", comment.char = "", header=T, stringsAsFactors=F)

### edit data table
fcount_SE_mrna_Ts_001 = trim_dataset(fcount_SE_mrna_Ts_001)
head(fcount_SE_mrna_Ts_001)

##### >> featureCounts mt results

In [ ]:
fcount_SE_mt_Ts_001 = read.table(paste0(workdir, "/summary_featurecounts_results_gencode_mt_transcript.txt"), sep="\t", comment.char = "", header=T, stringsAsFactors=F)

### edit data table
fcount_SE_mt_Ts_001 = trim_dataset(fcount_SE_mt_Ts_001)
head(fcount_SE_mt_Ts_001)

##### >> featureCounts rRNA results

In [ ]:
fcount_SE_rrna_Ts_001 = read.table(paste0(workdir, "/summary_featurecounts_results_gencode_rrna_transcript.txt"), sep="\t", comment.char = "", header=T, stringsAsFactors=F)

### edit data table
fcount_SE_rrna_Ts_001 = trim_dataset(fcount_SE_rrna_Ts_001)
head(fcount_SE_rrna_Ts_001)

##### >> featureCounts mRNA (gene-level) results

In [ ]:
fcount_SE_mrna_gene_001 = read.table(paste0(workdir, "/summary_featurecounts_results_gencode_mrna_gene.txt"), sep="\t", comment.char = "", header=T, stringsAsFactors=F)

### edit data table
fcount_SE_mrna_gene_001 = trim_dataset(fcount_SE_mrna_gene_001)
head(fcount_SE_mrna_gene_001)

##### >> Calculating RPKM from featureCounts results

In [ ]:
fcount_SE_merge_001 = read.table(paste0(workdir, "/mergefcounts_gencode_mrna_gene.txt"), sep="\t", comment.char = "", header=T, stringsAsFactors=F)

dim(fcount_SE_merge_001)

In [ ]:
gene_length = data.frame(Geneid=fcount_SE_merge_001$Geneid, length=fcount_SE_merge_001$Length, stringsAsFactors=F)
head(gene_length)

### check "ERCC lengh" contains
dim(subset(gene_length, grepl("ERCC", Geneid)))

In [ ]:
fcount_SE_merge_001_rpkm_set = calc_rpkm_counts(fcount_SE_merge_001, gene_length, exclude_name=exclude_samplename)

dim(fcount_SE_merge_001_rpkm_set$rpkm)
dim(fcount_SE_merge_001_rpkm_set$rpkm_log)

In [ ]:
fcount_SE_merge_001_rpkm_gene = fcount_SE_merge_001_rpkm_set$rpkm[!grepl("ERCC", rownames(fcount_SE_merge_001_rpkm_set$rpkm)),]
fcount_SE_merge_001_rpkm_gene_log = log10(fcount_SE_merge_001_rpkm_gene+1)

dim(fcount_SE_merge_001_rpkm_gene)
dim(fcount_SE_merge_001_rpkm_gene_log)

## 2. Plots of QC results
In this section, we will make plots of QC results.

### Fig. 1: FastQC results (All cells)

#### >> The total number of sequenced reads

In [ ]:
g1 = plot_totalseq(fastqc_SE_001, "fastq_totalseq", outdir)

options(repr.plot.width=10, repr.plot.height=4)
plot_grid (g1)

#### >> Average of %GC of sequenced reads

In [ ]:
g1 = plot_perGC(fastqc_SE_001, "fastq_perGC", outdir)

options(repr.plot.width=10, repr.plot.height=4)
plot_grid (g1)

### Fig. 2: Read Distribution ('blacklist' samples (cells) are excluded)

#### >> Rate of reads mapped to genome

In [ ]:
g1 = plot_assignedgene_rate(fastqc_SE_001, readDist_SE_001, "fastq_genome_assigned_rate", outdir, 100, nonplot=exclude_samplename)

options(repr.plot.width=10, repr.plot.height=4)
plot_grid (g1)

####  >> Distribution of mapped reads over genomic features

For RamDA-seq, in addition to the CDS, 5UTR, and 3UTR fractions, intron fractions shows relatively high proportion. This is a feature of total RNA-seq methods.

In [ ]:
g1 = plot_readDist_summary(readDist_SE_001, "bam_readDistribution", outdir, nonplot=exclude_samplename)

options(repr.plot.width=6, repr.plot.height=7)
plot_grid(g1)

### Fig. 3: Gene body coverage ('blacklist' samples (cells) are excluded)
This figure shows mean read coverage over transcripts.

If the values from 5′ to 3′ are high and constant, it indicates the samples (cells) show full-length transcript coverage.

In [ ]:
g1 = plot_geneBodyCov_heatmap(geneBC_SE_001, "bam_geneBodyCoverage", "", outdir, nonplot=exclude_samplename)

options(repr.plot.width=5, repr.plot.height=4)
plot_grid(g1)

### Fig. 4: Strandness ('blacklist' samples (cells) are excluded)

This figure shows the proportion of reads classified as 'sense', 'antisense', or 'undetermined' acoording to the gene annotation.

The QC result varies depending on whether the experimental protocol is unstranded or stranded ones.

- If the experimental protocol is unstranded, the proportions of 'sense_fraction' **and** 'antisense_fraction' are similar.
- If the experimental protocol is stranded, the proportions of 'sense_fraction' **or** 'antisense_fraction' are dominant.

In [ ]:
g1 = plot_bar_inferexp(infer_SE_001, "bam_inferexperiment", outdir, nonplot=exclude_samplename)

options(repr.plot.width=10, repr.plot.height=4)
plot_grid (g1)

### Fig. 5: High sensitivity rrna mapping QC ('blacklist' samples (cells) are excluded)
This figure shows the proportion of reads mapped to rRNA genes.


In [ ]:
if (file.exists(highsensitivity_rrna_result_path)){
    
    g1 = plot_bar_highsenst_rrna(highsenst_rrna_SE_001, "bam_highsensitivity_rrnaQC", outdir, nonplot=exclude_samplename)

    options(repr.plot.width=10, repr.plot.height=4)
    plot_grid (g1)   
} else {
  cat("highsensitivity rrnaQC results not found.")
}

### Fig. 6: Percentage of reads assigned to genes ('blacklist' samples (cells) are excluded)
The below figures show the percentage of reads assigned to genes based on featureCounts results.

#### >> mRNA (gene level)

In [ ]:
g1 = plot_assigned_rate(fcount_SE_mrna_gene_001, "featurecounts_mRNA_gene", outdir, ylim=100, nonplot=exclude_samplename)

options(repr.plot.width=10, repr.plot.height=4)
plot_grid (g1)

#### >> mRNA (transcript level)

In [ ]:
g1 = plot_assigned_rate(fcount_SE_mrna_Ts_001, "featurecounts_mRNA_Ts", outdir, ylim=100, nonplot=exclude_samplename)

options(repr.plot.width=10, repr.plot.height=4)
plot_grid (g1)

#### >> mitocondria (transcript level)

In [ ]:
g1 = plot_assigned_rate(fcount_SE_mt_Ts_001, "featurecounts_mitocondria_Ts", outdir, ylim=100, nonplot=exclude_samplename)

options(repr.plot.width=10, repr.plot.height=4)
plot_grid (g1)

#### >> rRNA (transcript level)

In [ ]:
g1 = plot_assigned_rate(fcount_SE_rrna_Ts_001, "featurecounts_rRNA_Ts", outdir, ylim=100, nonplot=exclude_samplename)

options(repr.plot.width=10, repr.plot.height=4)
plot_grid (g1)

## 3. Plots of number of detected genes


#### >> Correlation of expression levels
Some samples (cells) are randomly chosen, and expression levels between cells are plotted.


In [ ]:
### plot detgenes num
options(repr.plot.width=6, repr.plot.height=6)
bar_detgene_001 = plot_detgene_bar(fcount_SE_merge_001_rpkm_gene, fcount_SE_merge_001_rpkm_gene_log, nonplot=exclude_samplename)

#### >> Number of detected genes

In [ ]:
options(repr.plot.width=10, repr.plot.height=4)
plot_grid(bar_detgene_001)

## 4. Plots of dimensionality reduction results
The below figures show the results of PCA, tSNE, umap.

In [ ]:
### dimentional reduction
fcount_SE_dimentrd_set = create_pca_tsne_umap(fcount_SE_merge_001_rpkm_gene_log, perplexity=10, local_connectivity=1.0, n_neighbors=15)

In [ ]:
plot_pca = ggplot_2D(fcount_SE_dimentrd_set[["pca_df"]], "PC1", "PC2", "pca plot", "", outdir)
plot_tsne = ggplot_2D(fcount_SE_dimentrd_set[["tsne_df"]], "V1", "V2", "tsne plot", "", outdir)
plot_umap = ggplot_2D(fcount_SE_dimentrd_set[["umap_df"]], "V1", "V2", "umap plot", "", outdir)

options(repr.plot.width=4, repr.plot.height=10)
plot_grid (plot_pca, plot_tsne, plot_umap, nrow = 3)